In [54]:
import board
import busio
import time

In [104]:
def get_GPS_data(device_address=0x42, device_register=0xFF, time_limit=60, verbose=False):
    """This function pulls data from a Ublox ZED-F9P and returns an array of NMEA messages
    
    First the function connects to the GPS via I2C at the specified address and interal
    register address. For the ZED-F9P the default device address is 0x42 and the default
    internal register is at 0xFF. After collecting the bytes from the device, the bytes
    are processed into readable char. Data collection starts once a $ is found and ends
    when the byte returned is equivalent to 255.
    
    This function takes between 0.3 seconds to 2 seconds to collect the data. The bottle
    neck is the data acquisition on the device and changing the code did not significantly
    impact run time.
    
    Args:
        device_address: a hex number representing the device address. The default value is
            0x42.
        device_register: a hex representing the interal data register address. The default
            value is 0xFF.
        time_limit: an int or float that puts a limit on how long this function will run
            before it quits collecting data. This is to prevent an infinite loop in case
            the device hangs up.
        verbose: a boolean specifying whether to print progress and time
            
    Returns:
        device_output: an array of strings representing NMEA messages
    """
    i2c_hook = busio.I2C(board.SCL, board.SDA)
    while not i2c_hook.try_lock():
        pass
    
    buffer = bytearray(1)
    data_initialized = False
    data_finished = False
    data_array = []
    start_time = time.perf_counter()
    run_time = 0
    while((not data_initialized or not data_finished) and run_time < time_limit):
        try:
            i2c_hook.writeto(device_address, bytes([device_register]))
            i2c_hook.readfrom_into(device, buffer)
            char_arr = [chr(x) for x in buffer]
            cur_char = char_arr[0]
            data_length = len(data_array)
            if(cur_char == '$' and not data_initialized):
                if verbose:
                    print("Data Collection Started")
                data_initialized = True
            if((data_initialized and int(buffer[0]) == 255) and data_length > 1): 
                if verbose:
                    print("Data Collection Finished")
                data_finished = True
            
            if(data_initialized and not data_finished):
                data_array.append(cur_char)
                
        except OSError:
            continue
        cur_time = time.perf_counter()
        run_time = float(cur_time - start_time)
        
    if verbose:
        print(run_time)
    data_output = "".join(raw_gps).split("$")
    data_output = data_output[1:]
    
    return data_output

In [105]:
raw_gps = get_GPS_data()
for message in raw_gps:
    print(message)

GNRMC,062756.00,A,3725.60084,N,12209.14757,W,0.493,,250620,,,A,V*0A

GNVTG,,T,,M,0.493,N,0.912,K,A*39

GNGGA,062756.00,3725.60084,N,12209.14757,W,1,08,1.19,37.6,M,-30.0,M,,*4C

GNGSA,A,3,01,03,04,22,,,,,,,,,2.60,1.19,2.31,1*0A

GNGSA,A,3,70,80,79,69,,,,,,,,,2.60,1.19,2.31,2*01

GNGSA,A,3,,,,,,,,,,,,,2.60,1.19,2.31,3*0E

GNGSA,A,3,,,,,,,,,,,,,2.60,1.19,2.31,4*09

GPGSV,2,1,08,01,53,237,23,03,53,319,25,04,28,267,23,11,20,213,,1*68

GPGSV,2,2,08,14,38,054,14,16,00,159,,17,02,306,,22,76,329,20,1*66

GPGSV,2,1,08,01,53,237,,03,53,319,,04,28,267,,11,20,213,,*5E

GPGSV,2,2,08,14,38,054,,16,00,159,,17,02,306,,22,76,329,,*50

GLGSV,3,1,09,69,33,072,26,70,53,346,20,71,17,299,,73,05,181,,1*74

GLGSV,3,2,09,79,36,049,20,80,49,132,25,84,03,202,,85,29,252,27,1*7F

GLGSV,3,3,09,86,23,312,,1*4E

GLGSV,3,1,09,69,33,072,,70,53,346,,71,17,299,,73,05,181,,*43

GLGSV,3,2,09,79,36,049,,80,49,132,,84,03,202,,85,29,252,,*4E

GLGSV,3,3,09,86,23,312,,*7F

GAGSV,1,1,00,*44

GAGSV,1,1,00,*44

GBGSV,1,1,00,*47

GB